In [ ]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
from scipy.signal import medfilt

In [ ]:
input_dir = './images'
filename = 'mercy.png'
output_folder = './stencils'

In [ ]:
def make_stencil(image, num_colours):
    # Convert the image to greyscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Flatten the image into shape (-1, 1)
    flattened_image = np.reshape(gray_image, newshape=(-1, 1))
    
    # Cluster the image into num_colours distinct colours
    kmeans = KMeans(n_clusters=num_colours, random_state=0).fit(flattened_image)
    kmeans.cluster_centers_
    
    # Recolour the image using the most different shades possible
    vfunc = np.vectorize(lambda index: int(kmeans.cluster_centers_[index]))
    
    # Map the recolouring function onto the image
    _image = vfunc(np.reshape(kmeans.labels_, newshape=gray_image.shape))
    
    return _image

In [ ]:
def linear_interp_colour_func(colour_1, colour_2):
    return lambda shade: colour_1 + ((colour_2 - colour_1) / 255.0) * shade


def recolour_image(image, colour_func):
    _image = np.zeros((image.shape[0], image.shape[1], 3), dtype=int)
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            _image[row][col] = colour_func(image[row][col])
    return _image.astype(np.uint8)

In [ ]:
# Read in the image in grayscale
image = cv2.imread(os.path.join(input_dir, filename))

name, extension = filename.split('.')

for num_colours in range(2, 10):
    # Make it into a stencil
    stencil = make_stencil(image, num_colours)

    # stencil = medfilt(stencil, 3)
    stencil = stencil.astype(np.uint8)

    if not os.path.exists(os.path.join(output_folder, name)):
        os.makedirs(os.path.join(output_folder, name))
    
    colour_func = linear_interp_colour_func(np.asarray([0, 255, 0]), np.asarray([0, 0, 0]))
    c_stencil = recolour_image(stencil, colour_func)
    
    # Save the final image with the colour function applied
    cv2.imwrite(
        os.path.join(output_folder, name, name + '_colour_' + str(num_colours)  + '.' + extension),
        c_stencil
    )
    
    # Save the final image
    cv2.imwrite(
        os.path.join(output_folder, name, name + '_' + str(num_colours) + '.' + extension), 
        stencil
    )
